<a href="https://colab.research.google.com/github/PendaGTP/Pokemining/blob/master/Pokeming_Extract_Basic_Picture_Information.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
from PIL import Image
import numpy
from sklearn.cluster import MiniBatchKMeans
import multiprocessing as mp
from functools import partial
from itertools import repeat
import os
import json
import time

'''
Quelques notes :

k-Means : 
 * cet algorithme permet de rechercher un nombre prédéterminé (NB_CLUSTER)
de grappes (appelé cluster) dans une image (dans notre application). 
 * le centre de chaque grappe est la moyenne arithmétique de tous les points appartenant à la grappe
'''
## Temps d'éxécutions en local avec 4 coeurs
# KMeans environ  : 160s pour 70 fichiers
# MiniBatchKMeans : 70s pour 70 fichiers

# Settings dataset folders
# Local
POKE_DATA_DIRECTORY = '../datasets/pokedata'
POKE_DATA_DIRECTORY_TEST = POKE_DATA_DIRECTORY+'/Abra'
# Google drive
GD_POKE_DATA_DIRECTORY = '/gdrive/My Drive/Colab Notebooks/pokemon-generation-one/dataset'
GD_POKE_DATA_DIRECTORY_TEST = GD_POKE_DATA_DIRECTORY+'/Abra'
GD_ROOT_NOTEBOOK = '/gdrive/My Drive/Colab Notebooks/'

# Nombre de valeur K-MEANS
NB_CLUSTER = 3


#########################################################################################################
# 
# Cette classe permet de récupérer le chemin (os) des fichiers d'un dossier et/ou de ses sous répertoires
#
class LocalDataset:
    subDirectories = [] # Contient le chemin de sous dossier
    files = []
    dictionnary = {}
    countFiles = 0
    
    def __init__(self,rootDirectory,subDirectory=None):
        self.rootDirectory = rootDirectory
        if subDirectory == None:
            self.setPathSubDirectory()
        else:
            self.subDirectories.append(subDirectory)
        self.setDictFolderFiles()
        
    def setPathSubDirectory(self):
        for directory in os.listdir(self.rootDirectory):
            self.subDirectories.append(self.rootDirectory+'/'+directory)
            
    def setDictFolderFiles(self):
        for subRep in self.subDirectories:
          name = subRep.split('/') 
          name = name[-1]
          self.dictionnary[name] = []
          for file in os.listdir(subRep):
              if (not(".svg" in file)):
                  self.dictionnary[name].append(subRep+"/"+file)
                  self.files.append(subRep+'/'+file)
                  self.countFiles +=1
    
    def getDictFolderFiles(self):
        return self.dictionnary

    def getSubDirectories(self):
        return self.subDirectories
    
    def getCountFiles(self):
        return self.countFiles
#
#
#########################################################################################################



#########################################################################################################
# 
# Cette classe d'enregistrer un dictionnaire dans un fichier json
#
class JSONWriter:
    def __init__(self, filePath):
        self.filePath = filePath

    def writeJSON(self, data):
        with open(self.filePath, 'w') as f:
            json.dump(data, f)
#
#
#########################################################################################################
 

           
#########################################################################################################
# 
# Cette classe permet gérer la création de nos images
#
class ImageManager:
    def __init__(self):
        self.images = []
    
    # Création d'une image et sauvegarde de sa référence
    def addImage(self, name, imgPath):
        self.images.append(ImageP(name,imgPath))
          
    def getImages(self):
        return self.images
#
#
#########################################################################################################   



#########################################################################################################
# 
# Cette classe représente nos images
#
class ImageP:

    def __init__(self, name, path):
        self.name = name # nom du pokemon -> nom du dossier
        self.path = path # chemin de l'image
        self.clusterCenters = {} # centre des NB_CLUSTER grappes
        self.format = None 
        self.size = None # size -> (width,height)

    
    def setKmeans(self,cluster_centers):
        self.clusterCenters["colors"]=[]
        for color in cluster_centers:
            self.clusterCenters["colors"].append({"red":color[0], "green":color[1], "blue":color[2]})
    
    def setFormat(self, imageFormat):
        self.format = imageFormat
    
    def setSize(self, size):
        self.size = size
    
    def toDict(self):
        imageDict = {}
        imageDict["name"]= self.name
        imageDict["path"] = self.path
        imageDict["colors"] = []
        
        for color in self.clusterCenters:
            imageDict["colors"].append({"red":color[0], "green":color[1], "blue":color[2]})

        imageDict["size"] = self.size
        imageDict["format"] = self.format
        
        return imageDict
        
        
        
#
#
#########################################################################################################



# Cette fonction calcule les k-means valeurs pour NB_CLUSTER grappes
def processingKmeans(image,return_list):
    print("-> Processing image : "+image.path)
    
    im = Image.open(image.path) # Création objet PIL
    
    image.setFormat(im.format) # Définit le format de l'objet image
    image.setSize(im.size) # Définit la taille de l'objet image
    
    
    numarray = numpy.array(im.getdata(), numpy.uint8) # Création array pixel rgb
    dim = numarray.ndim # Dimension de l'array
    
    if dim == 2: # clusters.fit n'accepte pas les images à 1 dimension
        clusters = MiniBatchKMeans(n_clusters = NB_CLUSTER)
        clusters.fit(numarray)
        image.setKmeans(clusters.cluster_centers_)
        
        return_list.append(image)
        print("Done process :"+image.path)
        

        
# Lancement du chronomètre
startTime = time.time()


# Initialisation des données
drive.mount('/gdrive') # Montage du google drive

dataset  = LocalDataset(GD_POKE_DATA_DIRECTORY,GD_POKE_DATA_DIRECTORY_TEST)
#dataset = LocalDataset(POKE_DATA_DIRECTORY,POKE_DATA_DIRECTORY_TEST)
allFiles = dataset.files


# Initialisation des objets images
imageManager = ImageManager()
images = []



for file in allFiles:
    pokemonName=file.split('/')
    imageManager.addImage(pokemonName[-1],file)

images = imageManager.getImages()
print("Nombre d'images à traiter : ",len(images))

  
# Initialisation du multiprocessing
processingManager = mp.Manager()
return_list_image = processingManager.list()
pool = mp.Pool(processes=4)


# Création / Lancement des processus
pool.starmap(processingKmeans, zip(images, repeat(return_list_image)))


# Préparation des données à exporter
exportData = []
for image in return_list_image:
    exportData.append(image.toDict())

# Export des données en JSON
jw = JSONWriter(GD_ROOT_NOTEBOOK+"data.json") # Fichier de sortie

print("Running -> Object to JSON ...")
jw.writeJSON(exportData)
print("Done -> Object to JSON ...")


# Arrêt du chronomètre
endTime = time.time()
totalTime = endTime - startTime
print('Total time: ', totalTime)



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
Nombre d'images à traiter :  42
-> Processing image : /gdrive/My Drive/Colab Notebooks/pokemon-generation-one/dataset/Abra/0282b2f3a22745f1a436054ea15a0ae5.jpg
-> Processing image : /gdrive/My Drive/Colab Notebooks/pokemon-generation-one/dataset/Abra/1788abb8b51f48509cfac8067bd99e14.jpg
-> Processing image : /gdrive/My Drive/Colab Notebooks/pokemon-generation-one/dataset/Abra/06b9eec4827d4d49b1b4c284308708df.jpg
-> Processing image : /gdrive/My Drive/Colab Notebooks/

In [21]:
!cat /proc/cpuinfo | grep physi

physical id	: 0
address sizes	: 46 bits physical, 48 bits virtual
physical id	: 0
address sizes	: 46 bits physical, 48 bits virtual
